In [1]:
import sys
import os
sys.path.append(os.path.abspath("Pytorch/"))
sys.path.append(os.path.abspath("models/"))

In [2]:
from XVA import *
from XVAFBSNNs import *

In [3]:
from FBSNNs import *
from CallOptionsBasket import *

In [4]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

M = 64  # number of trajectories (batch size)
N = 50  # number of time snapshots
D = 100 # number of dimensions
Mm = N ** (1/5)

layers = [D + 1] + 4 * [256] + [1]

Xi = np.array([1] * int(D))[None, :]
T = 1.0

"Available architectures"
mode = "Naisnet"  # FC and Naisnet are available
activation = "Sine"  # Sine, ReLU and Tanh are available
model = CallOptionsBasket(Xi, T, M, N, D, Mm, layers, mode, activation)

n_iter = 15 * 10**2
lr = 1e-3

In [5]:
model.load_model("models/100DCallOptionXVAPaper.pth")

In [6]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

M = 64  # number of trajectories (batch size)
N = 50  # number of time snapshots
D = 100 # number of dimensions
Mm = N ** (1/5)

layers = [1+1] + 4 * [256] + [1]

Xi = np.array([1] * int(D))[None, :]
T = 1.0

"Available architectures"
mode = "Naisnet"  # FC and Naisnet are available
activation = "Sine"  # Sine, ReLU and Tanh are available
xvamodel = XVA(Xi, T, M, N, D, Mm, layers, mode, activation, model)

n_iter = 15 * 10**2
lr = 1e-3

In [9]:
n_iter = 15 * 10**2
lr = 1e-5

In [8]:
tot = time.time()
print(xvamodel.device)
graph = xvamodel.train(n_iter, lr)
print("total time:", time.time() - tot, "s")

cuda:0
It: 0, Loss: 1.052e+03, Y0: 0.276, Time: 1.07, Learning Rate: 1.000e-03
It: 100, Loss: 9.236e-02, Y0: 0.070, Time: 30.42, Learning Rate: 1.000e-03
It: 200, Loss: 3.348e-02, Y0: 0.044, Time: 30.41, Learning Rate: 1.000e-03
It: 300, Loss: 8.060e-03, Y0: 0.025, Time: 30.62, Learning Rate: 1.000e-03
It: 400, Loss: 1.620e-03, Y0: 0.015, Time: 30.36, Learning Rate: 1.000e-03
It: 500, Loss: 1.057e-03, Y0: 0.012, Time: 30.51, Learning Rate: 1.000e-03
It: 600, Loss: 1.080e-03, Y0: 0.012, Time: 30.32, Learning Rate: 1.000e-03
It: 700, Loss: 5.414e-04, Y0: 0.011, Time: 30.83, Learning Rate: 1.000e-03
It: 800, Loss: 6.177e-04, Y0: 0.010, Time: 30.47, Learning Rate: 1.000e-03
It: 900, Loss: 4.696e-04, Y0: 0.010, Time: 30.34, Learning Rate: 1.000e-03
It: 1000, Loss: 3.825e-04, Y0: 0.010, Time: 30.13, Learning Rate: 1.000e-03
It: 1100, Loss: 2.712e-04, Y0: 0.011, Time: 30.09, Learning Rate: 1.000e-03
It: 1200, Loss: 3.666e-04, Y0: 0.011, Time: 30.46, Learning Rate: 1.000e-03
It: 1300, Loss: 1.

In [10]:
tot = time.time()
print(xvamodel.device)
graph = xvamodel.train(n_iter, lr)
print("total time:", time.time() - tot, "s")

cuda:0
It: 1400, Loss: 1.938e-04, Y0: 0.009, Time: 0.31, Learning Rate: 1.000e-05
It: 1500, Loss: 1.230e-04, Y0: 0.009, Time: 29.73, Learning Rate: 1.000e-05
It: 1600, Loss: 1.035e-04, Y0: 0.009, Time: 30.03, Learning Rate: 1.000e-05
It: 1700, Loss: 1.189e-04, Y0: 0.010, Time: 30.04, Learning Rate: 1.000e-05
It: 1800, Loss: 1.412e-04, Y0: 0.009, Time: 30.18, Learning Rate: 1.000e-05
It: 1900, Loss: 1.371e-04, Y0: 0.009, Time: 30.02, Learning Rate: 1.000e-05
It: 2000, Loss: 1.555e-04, Y0: 0.009, Time: 29.97, Learning Rate: 1.000e-05
It: 2100, Loss: 2.212e-04, Y0: 0.009, Time: 30.23, Learning Rate: 1.000e-05
It: 2200, Loss: 2.452e-04, Y0: 0.009, Time: 30.05, Learning Rate: 1.000e-05
It: 2300, Loss: 1.936e-04, Y0: 0.009, Time: 29.92, Learning Rate: 1.000e-05
It: 2400, Loss: 1.840e-04, Y0: 0.009, Time: 29.86, Learning Rate: 1.000e-05
It: 2500, Loss: 2.022e-04, Y0: 0.009, Time: 29.93, Learning Rate: 1.000e-05
It: 2600, Loss: 1.403e-04, Y0: 0.009, Time: 30.00, Learning Rate: 1.000e-05
It: 27

In [7]:
xvamodel.load_model("models/100DCallOptionCVADVAXVAPaper.pth")

In [8]:
t_test, W_test, C_test = xvamodel.fetch_minibatch()
X_pred, Y_pred = xvamodel.predict(C_test, t_test, W_test)

if type(t_test).__module__ != 'numpy':
    t_test = t_test.cpu().numpy()
if type(X_pred).__module__ != 'numpy':
    X_pred = X_pred.cpu().detach().numpy()
if type(Y_pred).__module__ != 'numpy':
    Y_pred = Y_pred.cpu().detach().numpy()

for i in range(63):
    t_test_i, W_test_i, C_test_i = xvamodel.fetch_minibatch()
    X_pred_i, Y_pred_i = xvamodel.predict(C_test_i, t_test_i, W_test_i)
    if type(X_pred_i).__module__ != 'numpy':
        X_pred_i = X_pred_i.cpu().detach().numpy()
    if type(Y_pred_i).__module__ != 'numpy':
        Y_pred_i = Y_pred_i.cpu().detach().numpy()
    if type(t_test_i).__module__ != 'numpy':
        t_test_i = t_test_i.cpu().numpy()
    X_pred = np.concatenate((X_pred, X_pred_i), axis=0)
    Y_pred = np.concatenate((Y_pred, Y_pred_i), axis=0)
    t_test = np.concatenate((t_test, t_test_i), axis=0)

In [9]:
Y_preds = Y_pred.reshape(4096, 51)

In [10]:
Y_preds[0,:][0]

0.008979328

In [13]:
xvamodel.save_model("models/100DCallOptionCVADVAXVAPaper.pth")